In [88]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
import kaggle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import featuretools as ft
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer, MultiLabelBinarizer


import os
import composeml as cp
import featuretools as ft
import dask.dataframe as dd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from dask.distributed import Client

### Importing and preparing the data

In [89]:
#Convert csv to DataFrame
oct19 = pd.read_csv("../data/2019-Oct.csv")
nov19 = pd.read_csv("../data/2019-Nov.csv")
dec19 = pd.read_csv("../data/2019-Dec.csv")
jan20 = pd.read_csv("../data/2020-Jan.csv")
feb20 = pd.read_csv("../data/2020-Feb.csv")

In [125]:
print(oct19.shape)
print(nov19.shape)
print(dec19.shape)
print(jan20.shape)
print(feb20.shape)
print(oct19.shape[0]+nov19.shape[0]+dec19.shape[0]+jan20.shape[0]+feb20.shape[0])

(4102283, 9)
(4635837, 9)
(3533286, 9)
(4264752, 9)
(4156682, 9)
20692840


In [126]:
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)
ecom_df.head()
print(ecom_df.shape)

(20692840, 9)


# Data Cleaning

### NAs and how to deal with them

In [127]:
#8757117 nas is Brand
#4598 nas in user_session
#20339246 nas in category_code DROPPED

In [128]:
#NAs in Brand
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)

In [129]:
#NAs in user_session 
ecom_df.dropna(inplace=True)

In [130]:
ecom_df.category_code.isna().sum()

0

In [131]:
ecom_df.category_code.value_counts()

appliances.environment.vacuum             149148
stationery.cartrige                        59712
apparel.glove                              52568
furniture.living_room.cabinet              30559
accessories.bag                            24251
furniture.bathroom.bath                    23928
appliances.personal.hair_cutter             5384
accessories.cosmetic_bag                    3644
appliances.personal.massager                3296
appliances.environment.air_conditioner       708
furniture.living_room.chair                  343
sport.diving                                   4
Name: category_code, dtype: int64

In [132]:
#Drop this column, as we will use category code instead
ecom_df.drop("category_id", axis=1, inplace = True)

In [133]:
#Get rid of negative prices
ecom_df = ecom_df[ecom_df['price']>= 0]

# Feature Engineering

purchase per session, group session, total of price added up for each purchase in session

### Time FE

In [134]:
ecom_df['event_time'] = pd.to_datetime(ecom_df['event_time'])

In [135]:
ecom_df['month'] = pd.DatetimeIndex(ecom_df['event_time']).month
ecom_df['year'] = pd.DatetimeIndex(ecom_df['event_time']).year
ecom_df['hour'] = pd.DatetimeIndex(ecom_df['event_time']).hour
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour
21,2019-10-01 00:00:53+00:00,view,5856191,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0,10,2019,0
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0


In [ ]:
#convert year to string
ecom_df.year = ecom_df["year"].to_string()
ecom_df["year"].dtype

In [136]:
ecom_df['day'] = ecom_df['event_time'].dt.dayofweek
ecom_df['day'] = ecom_df['day'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

In [137]:
when_df = pd.DataFrame({'hour':range(1, 25)})
bins = [0,6,12,18,24]
labels = ['Night', 'Morning','Afternoon','Evening']
when_df['when'] = pd.cut(when_df['hour'], bins=bins, labels=labels, include_lowest=True)
#fix bellow
ecom_df = ecom_df.join(when_df['when'], on="hour")

In [138]:
ecom_df['month'] = ecom_df['month'].replace({1: 'January', 2: 'Febuary', 10: 'October', 11: 'November', 12: 'December'})

In [139]:
ecom_df.sample()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour,day,when
298093,2019-10-02 18:07:47+00:00,view,5855507,appliances.environment.vacuum,max,79.21,519238521,f1d0e341-5070-454d-afe8-223c607fa6ae,October,2019,18,Wednesday,Evening


In [140]:
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour,day,when
21,2019-10-01 00:00:53+00:00,view,5856191,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0,October,2019,0,Tuesday,Night
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night


In [141]:
ecom_df["current_session_clicks"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

DataError: No numeric types to aggregate

In [ ]:
#need to turn the features into numeric for bellow to work
ecom_df["current_session_event_types"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

# Labeling

In [142]:
encoder = LabelBinarizer()
etype_labels = pd.DataFrame(encoder.fit_transform(ecom_df['event_type']), columns=encoder.classes_, index=ecom_df.index)
etype_labels.head()

,cart,purchase,remove_from_cart,view
21,0,0,0,1
264,0,0,1,0
269,0,0,1,0
274,0,0,1,0
276,0,0,1,0


In [143]:
encoder = LabelBinarizer()
brand_labels = pd.DataFrame(encoder.fit_transform(ecom_df['brand']), columns=encoder.classes_, index=ecom_df.index)
brand_labels.head()

,babyliss,benovy,concept,cutrin,depilflax,dewal,domix,emil,entity,estel,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
276,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
encoder = LabelBinarizer()
productid_labels = pd.DataFrame(encoder.fit_transform(ecom_df['product_id']), columns=encoder.classes_, index=ecom_df.index)
productid_labels.head()

,4921,4923,4934,4962,4965,4983,4986,5395,6736,6737,...,5927895,5928193,5928194,5928195,5928196,5928359,5928360,5929649,5929650,5929651
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
ohe = OneHotEncoder(sparse=False, drop='first')
catco_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['category_code']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["category_code"]].columns))
catco_labels.head()

,category_code_accessories.cosmetic_bag,category_code_apparel.glove,category_code_appliances.environment.air_conditioner,category_code_appliances.environment.vacuum,category_code_appliances.personal.hair_cutter,category_code_appliances.personal.massager,category_code_furniture.bathroom.bath,category_code_furniture.living_room.cabinet,category_code_furniture.living_room.chair,category_code_sport.diving,category_code_stationery.cartrige
21,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [146]:
ohe = OneHotEncoder(sparse=False, drop='first')
day_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['day']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["day"]].columns))
day_labels.head()

,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
21,0.0,0.0,0.0,0.0,1.0,0.0
264,0.0,0.0,0.0,0.0,1.0,0.0
269,0.0,0.0,0.0,0.0,1.0,0.0
274,0.0,0.0,0.0,0.0,1.0,0.0
276,0.0,0.0,0.0,0.0,1.0,0.0


In [147]:
ohe = OneHotEncoder(sparse=False, drop='first')
month_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['month']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["month"]].columns))
month_labels.head()

,month_Febuary,month_January,month_November,month_October
21,0.0,0.0,0.0,1.0
264,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,1.0


In [162]:
ohe = OneHotEncoder(sparse=False, drop='first')
when_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['when']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["when"]].columns))
when_labels.head()

,when_Evening,when_Morning,when_Night
66,0.0,0.0,1.0
67,0.0,0.0,1.0
68,0.0,0.0,1.0
69,0.0,0.0,1.0
264,0.0,0.0,1.0


In [ ]:
ohe = OneHotEncoder(sparse=False, drop='first')
year_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['year']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["year"]].columns))
year_labels.head()

In [148]:
#Join our Labeled Columns to our main DataFrame
ecom_df = ecom_df.join(etype_labels)
ecom_df = ecom_df.join(brand_labels)
ecom_df = ecom_df.join(productid_labels)
ecom_df = ecom_df.join(catco_labels)
ecom_df = ecom_df.join(day_labels)
ecom_df  = ecom_df.join(month_labels)
ecom_df  = ecom_df.join(when_labels)                        
ecom_df  = ecom_df.join(year_labels)
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,...,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October
21,2019-10-01 00:00:53+00:00,view,5856191,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0,October,2019,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
33,2019-12-01 00:01:47+00:00,view,5732026,furniture.bathroom.bath,domix,4.16,509445281,91e9d610-759a-4831-b89a-22f2ab5fbc0f,December,2019,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,2019-12-01 00:02:42+00:00,view,5746846,furniture.bathroom.bath,irisk,52.70,579970581,0b92a801-25d8-4da5-b514-fda26db3faae,December,2019,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,2019-11-01 00:04:33+00:00,purchase,5766980,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,2019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
67,2019-11-01 00:04:33+00:00,purchase,5767494,stationery.cartrige,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,2019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Purchases_df

Created a dataframe of just purchases to do simplier features

In [ ]:
purchases_df = ecom_df.loc[ecom_df.event_type=='purchase']
purchases_df.shape

In [ ]:
#add column that adds total value of purchases for that user_session
purchases_df["session_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_session)["price"].cumsum()
#ecom_df['duplicate'] = ecom_df.groupby(cols)['user_id'].transform('size')
#can use cumsum to generate up to this point sums by user ID
purchases_df["lifetime_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_id)["price"].cumsum()
#purchases from session
purchases_df["purchases_in_session"] = purchases_df.groupby(purchases_df.user_id)["user_session"].transform('count')
#add event_type numericall
purchases_df["event_type"]=0
purchases_df.head()

### Removed_df

Created a dataframe of just removed to do simplier features

In [ ]:
removed_df = ecom_df.loc[ecom_df.event_type=="remove_from_cart"]
removed_df["event_type"]=1
removed_df.head()

# LETS TRY AN FSM SHRUGS

10/17 10pm update theres an issue with the merge thats causing the classes to become a single class. Will fix this tomorrow zZz

In [149]:
temp_df = ecom_df[ecom_df.event_type != 'view']
temp_df = temp_df[temp_df.event_type != 'cart']
print(temp_df.event_type.value_counts())
print(temp_df.shape)

remove_from_cart    242163
purchase            104238
Name: event_type, dtype: int64
(346401, 707)


In [150]:
#purchases_df.drop("event_type", axis=1, inplace=True)
#removed_df.drop("event_type", axis=1, inplace=True)
dataframes=[purchases_df, removed_df]
fsm = pd.concat(dataframes)

NameError: name 'purchases_df' is not defined

In [ ]:
#quick nans fix for missing columns from removed_df
fsm['session_cumulative_purchases'] = fsm['session_cumulative_purchases'].fillna(0)
fsm['purchases_in_session'] = fsm['purchases_in_session'].fillna(0)
fsm['lifetime_cumulative_purchases'] = fsm['lifetime_cumulative_purchases'].fillna(0)
fsm.sample(10)

In [ ]:
#HERE WE ARE LOOSING ALL MY CLASSES
le = preprocessing.LabelEncoder()

le.fit(ecom_df['event_type'])

ecom_df['event_type']=le.transform(ecom_df['event_type'])

ecom_df.event_type.value_counts()

# Models

### FSM

This is a proof of concept not I will spend my time feature engineering to make this model better. I have started some of it above.

In [151]:
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = ecom_df[ecom_df.event_type != 'cart']
ecom_df.sample()


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,...,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October
1239766,2020-01-11 14:33:24+00:00,remove_from_cart,5901858,apparel.glove,benovy,5.62,451861530,ddd2ea30-47f5-4e93-b770-8fedc72ca57e,January,2020,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [157]:
#"total_visits", "total_purchases_session"
X = ecom_df.drop(["purchase", "month", "year", "hour", "day", "when", "user_id", "product_id", "category_code", "cart", "purchase", "remove_from_cart", "view", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type

In [158]:
X.sample(20)
#why is their cumulative purchases in session in where nothing was bought

,price,babyliss,benovy,concept,cutrin,depilflax,dewal,domix,emil,entity,...,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October
608337,10.16,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2362765,10.16,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1994866,30.95,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
681353,5.24,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
556131,1.98,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3520239,10.16,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2851330,3.41,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
318148,48.41,0,0,0,0,0,0,0,1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3937266,79.21,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3377462,7.16,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [160]:
import lightgbm as lgb

In [161]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train, y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train)
test_preds_lgbm = lgbm.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.90      0.23      0.36     20694
remove_from_cart       0.75      0.99      0.85     48587

        accuracy                           0.76     69281
       macro avg       0.83      0.61      0.61     69281
    weighted avg       0.80      0.76      0.71     69281



# Visuals

### Graph 1

In [30]:
user_df = ecom_df[['user_id','event_type','price']]

In [33]:
graph_df = user_df[['user_id','price']]
graph_df = graph_df.groupby(['user_id'])['price'].sum().reset_index()
graph_df = graph_df.sort_values(by=['price'],ascending=False)
graph_df = graph_df.rename(columns={'price':'spent'})

In [37]:
price_list = graph_df['spent'].to_list()
plt.hist(x= price_list,bins=100,log=True, color='green')
plt.xlabel('Spent amount')
plt.ylabel('Customer Count')
plt.title('Customer Spending')
plt.savefig("Customer_Spend")

### Graph 2

In [47]:
# Daily visitors number
date = ecom_df.loc[:,['event_time','user_id']]
visitor_by_date = date.groupby(['event_time'])['user_id'].nunique()
visitor_by_date

event_time
2019-10-01 00:00:53+00:00    1
2019-10-01 00:07:58+00:00    1
2019-10-01 00:08:03+00:00    1
2019-10-01 00:08:08+00:00    1
2019-10-01 00:08:10+00:00    1
                            ..
2020-02-29 23:47:54+00:00    1
2020-02-29 23:51:55+00:00    1
2020-02-29 23:54:28+00:00    1
2020-02-29 23:54:43+00:00    1
2020-02-29 23:54:44+00:00    1
Name: user_id, Length: 330514, dtype: int64